In [ ]:
import pandas as pd
import numpy as np
import difflib
import googlemaps


residences = pd.DataFrame(columns=['address', 'zipcode', 'latitude', 'longitude'])

survivors = pd.DataFrame(columns=['name'])

contact_info = pd.DataFrame(columns=['survivor_id', 'contact_info'])

contact_points = pd.DataFrame(columns = ['residence_id', 'survivor_id', 'date', 
                                         'water_lvl', 'damage_lvl', 'notes1', 'notes2', 
                                         'notes3', 'intaker'])



In [2]:
keys = pd.read_csv('../data/API.txt')
gmap_key = keys.loc[keys['API']=='Google Maps', 'Key'].values[0]
gmaps = googlemaps.Client(key=gmap_key)

def find_or_add_residence(address):
    global residences
    try:
        geo = gmaps.geocode(address+', TN')
        lat = geo[0]['geometry']['location']['lat']
        lon = geo[0]['geometry']['location']['lng']
        zipcode = geo[0]['address_components'][-1]['long_name']

        coord_search = residences[(residences['latitude']==lat) & (residences['longitude']==lon)]

        if coord_search.shape[0] == 0:
            residences = residences.append({'address':address, 'latitude':lat, 
                                            'longitude':lon, 'zipcode':zipcode},
                                           ignore_index=True)
            res_id = residences.shape[0] - 1
            return res_id 
        else:
            res_id = coord_search.index.values[0]
            return res_id
    except:
        print(address)
        return address

In [3]:
def find_or_add_survivor(**kwargs):
    global survivors
    global contact_info
    
    name = kwargs['name'].upper().strip()
    name_search = survivors[survivors['name']==name]
    
    if name_search.shape[0] != 0:
        surv_id = name_search.index.values[0]
        
    elif len(difflib.get_close_matches(name, survivors['name'], n=1, cutoff=0.90)) > 0:
        match = difflib.get_close_matches(name, survivors['name'], n=1, cutoff=0.90)[0]
        surv_id = survivors.loc[survivors['name']==match].index.values[0]
    
    else:
        survivors = survivors.append({'name':name},ignore_index=True)
        surv_id = survivors.shape[0] - 1
    
    for k,v in kwargs.items():
        if k != 'name' and v.strip() != '':
            contact_info = contact_info.append({'survivor_id':surv_id, 'contact_info':v}, ignore_index=True)
    
    return surv_id
        

In [4]:
def create_contact_point(**kwargs):
    global contact_points
    contact_points = contact_points.append(kwargs, ignore_index=True)

    

nash_codes = pd.read_csv('../data/CodesDamageAssessment_4_15_21.csv', parse_dates=['CreationDate'], 
                         dtype={'Address':'object','Extent of Damage':'object','Type of Structure':'object',
                                'Water Level':'float64','Note':'object','Creator':'object'})
                                
nash_codes = nash_codes.dropna(subset=['Address'])
nash_codes = nash_codes.drop_duplicates(ignore_index=True)

mask = (nash_codes['Type of Structure']=='Residential') & (nash_codes['Extent of Damage'] != 'None')
for ind, row in nash_codes[mask].iterrows():
    add = row['Address'].upper().strip()
    res_id = find_or_add_residence(address=add)
    surv_id = None
    create_contact_point(residence_id = res_id, survivor_id = surv_id, 
                         date = row['CreationDate'], water_lvl = row['Water Level'], 
                         damage_lvl = row['Extent of Damage'], notes1=row['Notes'], 
                         intaker='Nashville Codes: ' + row['Creator'])

oem = pd.read_csv('../data/OEM_4_12_21.csv', 
                         dtype={'PropHouse':'object','PropZip':'object','PropStreet':'object',
                                'Extent of Damage':'object','Water Level':'object','Notes':'object',
                                'Damage 2':'object'})

oem = oem.fillna('')
oem = oem.drop_duplicates(ignore_index=True)

mask = oem['Extent of Damage'] != 'None'
for ind, row in oem[mask].iterrows():
    add = row['PropHouse'].upper().strip() + ' ' + row['PropStreet'].upper().strip()
    res_id = find_or_add_residence(address=add)
    surv_id = None
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-04-12'), 
                         water_lvl = row['Water Level'], damage_lvl = row['Extent of Damage'], 
                         notes1='NOTES: '+row['Notes'], notes2='DAMAGE 2: ' + row['Damage 2'], intaker='OEM')

nerve_4_9 = pd.read_csv('../data/NERVE_040921.csv', parse_dates=[10],
                         dtype={'Street No.':'object','Street':'object','First Name':'object','Last Name':'object',
                                'Email':'object','Phone Number':'object','Structure Damage Level':'object',
                                'Property Damage Described':'object','Comments':'object',
                                'Other - Property Damage Described':'object'})
nerve_4_9 = nerve_4_9.fillna('')
nerve_4_9 = nerve_4_9.drop_duplicates(ignore_index=True)

for ind, row in nerve_4_9.iterrows():
    add = row['Street No.'].upper().strip() + ' ' + row['Street'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['First Name'] + ' ' + row['Last Name'].upper().strip()
    
    if name == ' ':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone Number'].strip(), contact_2 = row['Email'].strip())
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = row['CreationDate'], 
                         damage_lvl = row['Structure Damage Level'], intaker='NERVE: Self Reported',
                         notes1='Property Damage Described: ' + row['Property Damage Described'], 
                         notes2 = 'Comments:' + row['Comments'], 
                         notes3 = 'Other - Property Damage Described: ' + row['Other - Property Damage Described'])

canvass_3_30 = pd.read_csv('../data/canvass_3_30.csv',
                         dtype={'Street Number':'object','Street Name':'object','Name':'object',
                                'Language Preference':'object','Saturday Help':'object',
                                'Phone Number':'object','Notes':'object'})
canvass_3_30 = canvass_3_30.fillna('')
canvass_3_30 = canvass_3_30.drop_duplicates(ignore_index=True)

for ind, row in canvass_3_30.iterrows():
    add = row['Street Number'].upper().strip() + ' ' + row['Street Name'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['Name']
    if name == '':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone Number'].strip())
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-03-31'),
                         intaker='Canvass Volunteer from Hands On Nashville',
                         notes1= 'Language Preference: ' + row['Language Preference'], 
                         notes2 = 'Notes' + row['Notes'], 
                         notes3 ='Followup: ' + row['Saturday Help'])

canvass_4_2 = pd.read_csv('../data/canvass_4_2.csv', parse_dates=[2],
                         dtype={'What is the house number?':'object','What is the street name?':'object',
                                'Is there visible debris on the property?':'object','Resident Name':'object',
                                'Resident phone number':'object','Language?':'object',
                                'Was the home impacted? (Either can you see this visually or did the resident disclose?)':'object',
                                'Does the resident want volunteers to assist with response?':'object',
                                'Does the resident need any of the following?':'object'})

canvass_4_2 = canvass_4_2.fillna('')
canvass_4_2 = canvass_4_2.drop_duplicates(ignore_index=True)


mask = canvass_4_2['Was the home impacted? (Either can you see this visually or did the resident disclose?)'] == 'Yes'
for ind, row in canvass_4_2[mask].iterrows():
    add = row['What is the house number?'].upper().strip() + ' ' + row['What is the street name?'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['Resident Name']
    if name == '':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Resident phone number'].strip())
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = row['Completion time'],
                         intaker='Canvass Volunteer from Hands On Nashville',
                         notes1= 'Language: ' + row['Language?'], 
                         notes2 = 'Requested Assistance: ' + row['Does the resident want volunteers to assist with response?'], 
                         notes3 ='Specific Needs: ' + row['Does the resident need any of the following?'])

nash_responds = pd.read_csv('../data/nash_responds_4_6.csv',
                         dtype={'First Name':'object','Last Name':'object','Address 1':'object',
                                'Email':'object','Phone':'object','Request Details':'object'})
nash_responds = nash_responds.drop_duplicates(ignore_index=True)



for ind, row in nash_responds.iterrows():
    add = row['Address 1'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['First Name'] + ' ' + row['Last Name']
    
    if name == ' ':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone'].strip(), contact_2 = row['Email'])
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-04-06'), 
                         intaker='Nashville Responds: Self Reported',
                         notes1='Request Details: ' + row['Request Details'])

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

survivors.to_csv('../data/survivors_table.csv')
residences.to_csv('../data/residences_table.csv')
contact_info.to_csv('../contact_info_table.csv')
contact_points.to_csv('../data/contact_points_table.csv')

In [5]:
residences = pd.read_csv('../data/residences_table.csv', usecols=[1,2,3,4])
survivors = pd.read_csv('../data/survivors_table.csv', usecols=[1])
contact_info = pd.read_csv('../data/contact_info_table.csv',usecols=[1,2], dtype={'survivor_id':'object'})
contact_points = pd.read_csv('../data/contact_points_table.csv', usecols=[1,2,3,4,5,6,7,8,9], 
                             dtype={'residence_id':'object', 'survivor_id':'object'})

In [6]:
print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

CONTACT POINTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   residence_id  1152 non-null   object
 1   survivor_id   300 non-null    object
 2   date          1152 non-null   object
 3   water_lvl     272 non-null    object
 4   damage_lvl    645 non-null    object
 5   notes1        809 non-null    object
 6   notes2        543 non-null    object
 7   notes3        301 non-null    object
 8   intaker       1152 non-null   object
dtypes: object(9)
memory usage: 81.1+ KB

RESIDENCES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    745 non-null    object 
 1   zipcode    745 non-null    object 
 2   latitude   745 non-null    float64
 3   longitude  745 non-null    float64
dtypes: float64(2), 

(None, None, None, None, None, None, None, None)

In [7]:
nerve_3_30 = pd.read_csv('../data/nerve_3_30.csv',
                         dtype={'Street Number.':'object','St (Apt)':'object','First':'object','Last':'object',
                                'Email':'object','Phone':'object','Damage 1':'object',
                                'Damage 2':'object','Column1':'object',})

nerve_3_30 = nerve_3_30.fillna('')
nerve_3_30 = nerve_3_30.drop_duplicates(ignore_index=True)

for ind, row in nerve_3_30.iterrows():
    add = row['Street Number'].upper().strip() + ' ' + row['St (Apt)'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['First'].upper().strip() + ' ' + row['Last'].upper().strip()
    
    if name == ' ':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone'].strip(), contact_2 = row['Email'].strip())
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-3-30'), 
                         damage_lvl = row['Column1'], intaker='NERVE: Self Reported',
                         notes1='Damage1: ' + row['Damage 1'], 
                         notes2 = 'Damage2: ' + row['Damage 2'])

140 CHEROKEE HILLS DR
50053 MITCHELL DR


In [8]:
print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

CONTACT POINTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   residence_id  1790 non-null   object
 1   survivor_id   927 non-null    object
 2   date          1790 non-null   object
 3   water_lvl     272 non-null    object
 4   damage_lvl    1283 non-null   object
 5   notes1        1447 non-null   object
 6   notes2        1181 non-null   object
 7   notes3        301 non-null    object
 8   intaker       1790 non-null   object
dtypes: object(9)
memory usage: 126.0+ KB

RESIDENCES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    862 non-null    object 
 1   zipcode    862 non-null    object 
 2   latitude   862 non-null    float64
 3   longitude  862 non-null    float64
dtypes: float64(2),

(None, None, None, None, None, None, None, None)

In [9]:
metro_water_4_12 = pd.read_csv('../data/metro_water_4_12.csv',
                         dtype={'Address':'object','Contact Name':'object','Email':'object',
                                'Phone':'object','Inspection Notes':'object'})

metro_water_4_12 = metro_water_4_12.fillna('')

for ind, row in metro_water_4_12.iterrows():
    add = row['Address'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['Contact Name'].upper().strip()
    
    if name == '':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone'].strip(), contact_2 = row['Email'].strip())
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-4-12'), 
                         intaker='Metro Water Inspector', notes1='Inspection Notes: ' + row['Inspection Notes'])

In [10]:
print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

CONTACT POINTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802 entries, 0 to 1801
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   residence_id  1802 non-null   object
 1   survivor_id   936 non-null    object
 2   date          1802 non-null   object
 3   water_lvl     272 non-null    object
 4   damage_lvl    1283 non-null   object
 5   notes1        1459 non-null   object
 6   notes2        1181 non-null   object
 7   notes3        301 non-null    object
 8   intaker       1802 non-null   object
dtypes: object(9)
memory usage: 126.8+ KB

RESIDENCES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    869 non-null    object 
 1   zipcode    869 non-null    object 
 2   latitude   869 non-null    float64
 3   longitude  869 non-null    float64
dtypes: float64(2),

(None, None, None, None, None, None, None, None)

In [14]:
hotline = pd.read_csv('../data/hotline.csv', parse_dates=[0],
                         dtype={'CC Status/Notes':'object','Name of person requesting help:':'object','Email address:':'object',
                                'Phone Number:':'object','Project Details':'object', 'Name of person filling out form:':'object',
                               'Type of help needed':'object', 'Language Spoken':'object', 'Address where help is needed:': 'object'})

hotline = hotline.fillna('')

for ind, row in hotline.iterrows():
    add = row['Address where help is needed:'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['Name of person requesting help:'].upper().strip()
    
    if name == '':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone Number:'].strip(), contact_2 = row['Email address:'].strip())
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = row['Timestamp'], 
                         intaker='Hotline: '+row['Name of person filling out form:'], 
                         notes1='Notes: ' + row['CC Status/Notes'], notes2='Details: '+row['Type of help needed'] + '\n' + row['Project Details'])

In [15]:
print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

CONTACT POINTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   residence_id  1999 non-null   object
 1   survivor_id   1033 non-null   object
 2   date          1999 non-null   object
 3   water_lvl     272 non-null    object
 4   damage_lvl    1283 non-null   object
 5   notes1        1656 non-null   object
 6   notes2        1378 non-null   object
 7   notes3        301 non-null    object
 8   intaker       1999 non-null   object
dtypes: object(9)
memory usage: 140.7+ KB

RESIDENCES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    884 non-null    object 
 1   zipcode    884 non-null    object 
 2   latitude   884 non-null    float64
 3   longitude  884 non-null    float64
dtypes: float64(2),

(None, None, None, None, None, None, None, None)

In [21]:
contact_points.sort_values(by='residence_id', ascending=False)

residence_id survivor_id                 date water_lvl damage_lvl  \
112            99         NaN  2021-04-01 21:32:00       NaN   affected   
362            98         NaN  2021-03-31 06:00:00       NaN      Major   
909            98          71  2021-04-02 17:30:00       NaN        NaN   
1050           98          71  2021-04-06 00:00:00       NaN        NaN   
1558           98          71  2021-03-30 00:00:00       NaN      minor   
...           ...         ...                  ...       ...        ...   
1584          101         486  2021-03-30 00:00:00       NaN      Minor   
113           100         NaN  2021-04-02 00:22:00       NaN      minor   
13             10         NaN  2021-04-02 01:02:00       NaN      minor   
1               1         NaN  2021-03-31 19:40:00       NaN      minor   
0               0         NaN  2021-04-01 02:02:00       NaN        NaN   

                                                 notes1  \
112        Property Damage Described: water_below,other   
362                                                 NaN   
909                                   Language: English   
1050  Request Details: Over 1 foot, bottom room seve...   
1558  Damage1: Flood Victim Assistance- Shelter, vol...   
...                                                 ...   
1584  Damage1: We have two needs:\n1. Our yard & dri...   
113             Property Damage Described: water_living   
13    Property Damage Described: water_below,water_l...   
1                Property Damage Described: water_below   
0                Property Damage Described: water_below   

                         notes2  \
112                   Comments:   
362                         NaN   
909   Requested Assistance: Yes   
1050                        NaN   
1558                  Damage2:    
...                         ...   
1584                  Damage2:    
113                   Comments:   
13                    Comments:   
1                     Comments:   
0                     Comments:   

                                                 notes3  \
112   Other - Property Damage Described: There is a ...   
362                                                 NaN   
909                             Specific Needs: Shelter   
1050                                                NaN   
1558                                                NaN   
...                                                 ...   
1584                                                NaN   
113                 Other - Property Damage Described:    
13    Other - Property Damage Described: Insulation ...   
1                   Other - Property Damage Described:    
0                   Other - Property Damage Described:    

                                                intaker  
112                                NERVE: Self Reported  
362   Nashville Codes: heidi.mariscal@nashville.gov_...  
909           Canvass Volunteer from Hands On Nashville  
1050                  Nashville Responds: Self Reported  
1558                               NERVE: Self Reported  
...                                                 ...  
1584                               NERVE: Self Reported  
113                                NERVE: Self Reported  
13                                 NERVE: Self Reported  
1                                  NERVE: Self Reported  
0                                  NERVE: Self Reported  

[1999 rows x 9 columns]

In [25]:
survivors.to_csv('../data/survivors_table.csv')
residences.to_csv('../data/residences_table.csv')
contact_info.to_csv('../data/contact_info_table.csv')
contact_points.to_csv('../data/contact_points_table.csv')